## Course - Manipulating Time Series Data in Python

# M4 Building a value-weighted index

## Select index components & import data

### Explore and clean company listing information
To get started with the construction of a market-value based index, you'll work with the combined listing info for the three largest US stock exchanges, the NYSE, the NASDAQ and the AMEX.
In this and the next exercise, you will calculate market-cap weights for these stocks.
We have already imported pandas as pd, and loaded the listings data set with listings information from the NYSE, NASDAQ, and AMEX. The column 'Market Capitalization' is already measured in USD mn.

Instructions

Inspect listings using .info().
Move the column 'Stock Symbol' into the index (inplace).
Drop all companies with missing 'Sector' information from listings.
Select companies with IPO Year before 2019.
Inspect the result of the changes you just made using .info().
Show the number of companies per 'Sector' using .groupby() and .size(). Sort the output in descending order.

In [ ]:
# Inspect listings
print(listings.info())

# Move 'stock symbol' into the index
listings.set_index('Stock Symbol', inplace=True)

# Drop rows with missing 'sector' data
listings.dropna(subset=['Sector'], inplace=True)

# Select companies with IPO Year before 2019
listings = listings[listings['IPO Year']<2019]

# Inspect the new listings data
print(listings.info())

# Show the number of companies per sector
print(listings.groupby('Sector').size().sort_values(ascending=False))


### Select and inspect index components
Now that you have imported and cleaned the listings data, you can proceed to select the index components as the largest company for each sector by market capitalization.
You'll also have the opportunity to take a closer look at the components, their last market value, and last price.

Instructions

We have already imported pandas as pd, and loaded the listings data with the modifications you made during the last exercise.
Use .groupby() and .nlargest() to select the largest company by 'Market Capitalization' for each 'Sector', and assign the result to components.
Print components, sorted in descending order by market cap.
Select Stock Symbol from the index of components, assign it to tickers and print the result.
Create a list info_cols that holds the column names Company Name, Market Capitalization, and Last Sale. Next, use .loc[] with tickers and info_cols to print() more details about the listings sorted in descending order by Market Capitalization).

In [ ]:
# Select largest company for each sector
components = listings.groupby(['Sector'])['Market Capitalization'].nlargest(1)

# Print components, sorted by market cap
print(components.sort_values(ascending=False))

# Select stock symbols and print the result
tickers = components.index.get_level_values('Stock Symbol')
print(tickers)

# Print company name, market cap, and last price for each component 
info_cols = ['Company Name', 'Market Capitalization', 'Last Sale']
print(listings.loc[tickers, info_cols].sort_values('Market Capitalization', ascending=False))

### Import index component price information
Now you'll use the stock symbols for the companies you selected in the last exercise to calculate returns for each company.

Instructions

We have already imported pandas as pd and matplotlib.pyplot as plt for you. We have also made the variable tickers available to you, which contains the Stock Symbol for each index component as a list.
Print tickers to verify the content matches your result from the last exercise.
Use pd.read_csv() to import 'stock_prices.csv', parsing the 'Date' column and also setting the 'Date' column as index before assigning the result to stock_prices. Inspect the result using .info().
Calculate the price return for the index components by dividing the last row of stock_prices by the first, subtracting 1 and multiplying by 100. Assign the result to price_return.
Plot a horizontal bar chart of the sorted returns with the title Stock Price Returns.

In [ ]:
# Print tickers
print(tickers)

# Import prices and inspect result
stock_prices = pd.read_csv('stock_prices.csv', parse_dates=['Date'], index_col='Date')
print(stock_prices.info())

# Calculate the returns
price_return = stock_prices.iloc[-1].div(stock_prices.iloc[0]).sub(1).mul(100)

# Plot horizontal bar chart of sorted price_return   
price_return.sort_values().plot(kind='barh', title='Stock Price Returns')
plt.show()


## Build a market-cap weighted index

### Calculate number of shares outstanding
The next step towards building a value-weighted index is to calculate the number of shares for each index component.

The number of shares will allow you to calculate the total market capitalization for each component given the historical price series in the next exercise.

Instructions

We have already imported pandas as pd, tickers and listings as in the previous exercises.

Inspect listings and print tickers.
Use .loc[] with the list of tickers to select the index components and the columns 'Market Capitalization' and 'Last Sale'; assign this to components.
Print the first five rows of components.
Create no_shares by dividing Market Capitalization by 'Last Sale'.
Print no_shares in descending order.

In [ ]:
# Inspect listings and print tickers
print(listings.info())
print(tickers)

# Select components and relevant columns from listings
components = listings.loc[tickers,['Market Capitalization','Last Sale']]

# Print the first rows of components
print(components.head())

# Calculate the number of shares here
no_shares = components['Market Capitalization'].div(components['Last Sale'])

# Print the sorted no_shares
print(no_shares.sort_values(ascending=False) )


#### Create time series of market value
You can now use the number of shares to calculate the total market capitalization for each component and trading date from the historical price series.

The result will be the key input to construct the value-weighted stock index, which you will complete in the next exercise.

Instructions

We have already imported pandas as pd and matplotlib.pyplot as plt for you. We have also created the variables components and stock_prices that you worked with in the last exercises.

Select the 'Number of Shares' from components, assign to no_shares, and print the result, sorted in the default (ascending) order.
Multiply stock_prices by no_shares to create a time series of market cap per ticker, and assign it to market_cap.
Select the first and the last row of market_cap and assign these to first_value and last_value.
Use pd.concat() to concatenate first_value and last_value along axis=1 and plot the result as horizontal bar chart.

In [ ]:
# Select the number of shares
no_shares = components['Number of Shares']
print(no_shares.sort_values())

# Create the series of market cap per ticker
market_cap = stock_prices.mul(no_shares)

# Select first and last market cap here
first_value = market_cap.iloc[0]
last_value = market_cap.iloc[-1]


# Concatenate and plot first and last market cap here
pd.concat([first_value,last_value],axis=1).plot(kind='barh')

plt.show()



### Calculate & plot the composite index
By now you have all ingredients that you need to calculate the aggregate stock performance for your group of companies.

Use the time series of market capitalization that you created in the last exercise to aggregate the market value for each period, and then normalize this series to convert it to an index.

Instructions

We have already imported pandas as pd and matplotlib.pyplot as plt for you. We have also loaded components and market_cap_series, which you worked with in the last exercise.

Aggregate the market cap per trading day by applying .sum() to market_cap_series with axis=1, assign to raw_index and print the result.
Normalize the aggregate market cap by dividing by the first value of raw_index and multiplying by 100. Assign this to index and print the result.
Plot the index with the title 'Market-Cap Weighted Index'.

In [ ]:
# Aggregate and print the market cap per trading day
raw_index = market_cap_series.sum(axis=1)
print(raw_index )

# Normalize the aggregate market cap here 
index = raw_index.div(raw_index.iloc[0]).mul(100)
print(index)

# Plot the index here
index.plot(title='Market-Cap Weighted Index')

plt.show()

## Evaluate index performance

### Calculate the contribution of each stock to the index
You have successfully built the value-weighted index. Let's now explore how it performed over the 2010-2016 period.

Let's also determine how much each stock has contributed to the index return.

Instructions

We have already imported pandas as pd and matplotlib.pyplot as plt for you. We have also loaded components and the index you worked with in the last exercise.

Divide the last index value by the first, subtract 1 and multiply by 100. Assign the result to index_return and print it.
Select the 'Market Capitalization' column from components.
Calculate the total market cap for all components and assign this to total_market_cap.
Divide the components' market cap by total_market_cap to calculate the component weights, assign it to weights, and print weights with the values sorted in default (ascending) order.
Multiply weights by the index_return to calculate the contribution by component, sort the values in ascending order, and plot the result as a horizontal bar chart.

In [ ]:
# Calculate and print the index return here
index_return = ((index.iloc[-1]/index.iloc[0])-1)*100
print(index_return)

# Select the market capitalization
market_cap = components['Market Capitalization']

# Calculate the total market cap
total_market_cap = market_cap.sum()

# Calculate the component weights, and print the result
weights = market_cap/total_market_cap
print(weights.sort_values())

# Calculate and plot the contribution by component
weights.mul(index_return).sort_values().plot(kind='barh')
plt.show()

### Compare index performance against benchmark I
The next step in analyzing the performance of your index is to compare it against a benchmark.

In the video, we used the S&P 500 as benchmark. You can also use the Dow Jones Industrial Average, which contains the 30 largest stocks, and would also be a reasonable benchmark for the largest stocks from all sectors across the three exchanges.

Instructions

We have already imported pandas as pd, matplotlib.pyplot as plt for you. We have also loaded your index and the DJIA data into variables index and djia, respectively, both as a pd.Series().

Convert index to a pd.DataFrame with the column name 'Index' and assign the result to data.
Normalize djia to start at 100 and add it as new column to data.
Show the total return for both index and djia by dividing the last row of data by the first, subtracting 1 and multiplying by 100.
Show a plot of both of the series in data.

In [ ]:
# Convert index series to dataframe here
data = index.to_frame('Index')

# Normalize djia series and add as new column to data
djia = djia.div(djia.iloc[0]).mul(100)
data['DJIA'] = djia

# Show total return for both index and djia
print(data.iloc[-1].div(data.iloc[0]).sub(1).mul(100))

# Plot both series
data.plot()
plt.show()


### Compare index performance against benchmark II
The next step in analyzing the performance of your index is to compare it against a benchmark.

In the video, we have use the S&P 500 as benchmark. You can also use the Dow Jones Industrial Average, which contains the 30 largest stocks, and would also be a reasonable benchmark for the largest stocks from all sectors across the three exchanges.

Instructions

We have already imported numpy as np, pandas as pd, matplotlib.pyplot as plt for you. We have also loaded your Index and the Dow Jones Industrial Average (normalized) in a variable called data.

Inspect data and print the first five rows.
Define a function multi_period_return that takes a numpy array of period returns as input, and returns the total return for the period. Use the formula from the video - add 1 to the input, pass the result to np.prod(), subtract 1 and multiply by 100.
Create a .rolling() window of length '360D' from data, and apply multi_period_return. Assign to rolling_return_360.
Plot rolling_return_360 using the title 'Rolling 360D Return'.

In [ ]:
# Inspect data
print(data.info())
print(data.head())

# Create multi_period_return function here
def multi_period_return(r):
    return (np.prod(r + 1) - 1) * 100

# Calculate rolling_return_360
rolling_return_360 = data.pct_change().rolling('360D').apply(multi_period_return)

# Plot rolling_return_360 here
rolling_return_360.plot(title='Rolling 360D Return')

plt.show()



## Index correlation & exporting to Excel

### Visualize your index constituent correlations
To better understand the characteristics of your index constituents, you can calculate the return correlations.

Use the daily stock prices or your index companies, and show a heatmap of the daily return correlations!

Instructions

We have already imported pandas as pd, matplotlib.pyplot as plt, and seaborn as sns. We have also loaded the historical price series of your index constituents into the variable stock_prices.

Inspect stock_prices using .info().
Calculate the daily returns for stock_prices and assign the result to returns.
Calculate the pairwise correlations for returns, assign them to correlations and print the result.
Plot a seaborn annotated heatmap of the daily return correlations with the title 'Daily Return Correlations'.

In [ ]:
# Inspect stock_prices here
print(stock_prices.info())

# Calculate the daily returns
returns = stock_prices.pct_change()

# Calculate and print the pairwise correlations
correlations = returns.corr()
print(correlations)

# Plot a heatmap of daily return correlations

sns.heatmap(correlations,annot=True)
plt.title('Daily Return Correlations')
plt.show()

### Save your analysis to multiple excel worksheets
Now that you have completed your analysis, you may want to save all results into a single Excel workbook.

Let's practice exporting various DataFrame to multiple Excel worksheets.

Instructions

We have already imported pandas as pd for you. We have also loaded both the historical price series of your index constituents into the variable stock_prices, and the index as index.

Inspect both index and stock_prices using .info().
Use .join() to combine index with stock_prices, and assign to data.
Apply .pct_change() to data and assign to returns.
Create pd.ExcelWriter and use with to export data and returns to excel with sheet_names of the same name.

In [ ]:
# Inspect index and stock_prices
print(index.info())
print(stock_prices.info())

# Join index to stock_prices, and inspect the result
data = stock_prices.join(index)
print(data.info())

# Create index & stock price returns
returns = data.pct_change() 

# Export data and data as returns to excel
with pd.ExcelWriter('data.xls') as writer:
    data.to_excel(excel_writer=writer, sheet_name='data')
    returns.to_excel(excel_writer=writer, sheet_name='returns')
